In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from pathlib import Path
import onnx
import onnxruntime
import coremltools
import time
import tf2onnx
import time
import os
import copy
import csv
import pandas as pd
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [2]:
max_features = 20000
maxlen = 300 # 80 #
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must 

25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 300)
x_test shape: (25000, 300)


In [3]:
framework = 'keras'
model_short_name = 'lstm'

onnx_path = '/Volumes/Cisco/Summer2022/onnx-exchange/Train2/conversion2/onnx/'
coreml_path = '/Volumes/Cisco/Summer2022/onnx-exchange/Train2/conversion2/coremltools/'
error_path = '/Volumes/Cisco/Summer2022/onnx-exchange/Train2/miss-classification/'

In [4]:
def convert_category(y):
    list_ = []
    for i in y:
        if i < 0.5:
            val = 0
        else:
            val = 1
        list_.append(val)
    return np.array(list_).astype(np.float32)
def model_scores(y_test, test_predict):
    return np.sum(y_test == test_predict)
def get_miss_classification(y1, y2):
    val = 0
    try:
        np.testing.assert_array_equal(y1, y2)
    except Exception as e:
        #print(e)
        for line_ in str(e).split('\n'):
            #print(' ---- : ', line_)
            if 'Mismatched elements' in line_:
                value = line_.replace('Mismatched elements: ', '').strip()
                val = int(value.split('(')[0].split('/')[0].replace(' ', ''))
                #print(val)
                #miss_perc_val_test_runtime = value[value.find("(")+1:value.find(")")]
                #print(value, perc_val)
                break
    return val

In [5]:
def to_onnx(x, y):
    
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
    #print("converting for batch: ", i)
    #y = np.argmax(y)
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    
    ### Original Model
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    try:
        with tf.device('/cpu:0'): 
            k_predict = model.predict(x)
    except Exception as e:
        print('Error keras: ')
        return None
    inference_time_original = time.time() - since_1
    y0 = convert_category(k_predict)
    #correct_original = np.sum(y0 == y)
    print(k_predict)
    print(y0, y)
    accuracy_original = model_scores(y, y0)
    miss_test_original = get_miss_classification(y, y0)
    
    print('accuracy_original: ',accuracy_original, miss_test_original)
    # ONNX Model
    
    t_elapsed_2 = time.time() - since_1
    #since_1 = time.time()
    #onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #load_time_onnx = time.time() - since_1
    #onnx.checker.check_model(onnx_model)
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    #ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: x}
    ort_outs = ort_session.run(None, ort_inputs)
    inference_time_onnx = time.time() - since_1
    # compare ONNX Runtime and PyTorch results
    #print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    ####### Mis-classification ONNX ######################################
    y2 = convert_category(ort_outs[0])
    #correct_onnx = np.sum(y2 == y)
    accuracy_onnx = model_scores(y, y2)
    miss_original_onnx = get_miss_classification(y0, y2)
    miss_test_onnx = get_miss_classification(y, y2)
    
    print('accuracy_onnx: ',accuracy_onnx, miss_original_onnx)
    
    abs_err = np.absolute(k_predict-ort_outs[0])
    rel_err = np.absolute(k_predict-ort_outs[0])/ np.absolute(ort_outs[0])
    ####### End of mis-classification ONNX ###################################### 
    
    
    ## CoreML
    
    #since_1 = time.time()
    #coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}.mlmodel".format(model_short_name, model_name))
    #load_time_coreml = time.time() - since_1
    
    #spec = coreml_model.get_spec()
    #coreml_model = coremltools.models.MLModel(spec)
    split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
    name_1 = split_[0].replace('name: "', '')
    name_1 = name_1.replace('"', '')
    
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:x})
    inference_time_coreml = time.time() - since_1
    ####### Mis-classification coreML ######################################
    y3 = convert_category(output_dict_test['Identity'])
    #correct_coreml = np.sum(y3 == y)
    accuracy_coreml = model_scores(y, y3)
    miss_original_coreml = get_miss_classification(y0, y3)
    miss_test_coreml = get_miss_classification(y, y3)
    print('accuracy_coreml: ',accuracy_onnx, miss_original_coreml)
    
    abs_err2 = np.absolute(k_predict-output_dict_test['Identity'])
    rel_err2 = np.absolute(k_predict-output_dict_test['Identity'])/ np.absolute(output_dict_test['Identity'])
        
    ####### End of mis-classification coreML ######################################
    #return correct_original,correct_onnx,correct_coreml 
    list_val = [accuracy_original, accuracy_onnx, accuracy_coreml, miss_original_onnx, miss_original_coreml,inference_time_original, inference_time_onnx, inference_time_coreml,abs_err, rel_err, abs_err2, rel_err2]
    return list_val

In [6]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def _lets_convert(x_test, y_test, data_writer_run):
    x_test = x_test.astype(np.float32)
    #y_test = x_test.astype(np.float32)
    list_val = to_onnx(x_test,y_test)
    accuracy_original = list_val[0]
    accuracy_onnx = list_val[1]
    accuracy_coreml = list_val[2]

    miss_original_onnx = list_val[3]
    miss_original_coreml = list_val[4]

    inference_time_original = list_val[5]
    inference_time_onnx = list_val[6]
    inference_time_coreml = list_val[7]
    
    mean_abs_onnx = np.mean(list_val[8])
    mean_abs_coreml = np.mean(list_val[10])
    mean_rel_onnx = np.mean(list_val[9])
    mean_rel_coreml = np.mean(list_val[11])
    std_abs_onnx = np.std(list_val[8])
    std_abs_coreml = np.std(list_val[10])
    std_rel_onnx = np.std(list_val[9])
    std_rel_coreml = np.std(list_val[11])
    
    total_ = np.sum(y_test == y_test)
    total_datasets = y_test.shape[0]
    
    
    print(list_val, total_, total_datasets)
        
    #time_elapsed = time.time() - since
    time_elapsed_overal_time = time.time() - since_orinal_0
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed_overal_time // 60, time_elapsed_overal_time % 60) )
    #print(correct_original, correct_onnx, correct_coreml, total, correct_original*100./total, precision2, recall2, f1)
    #return correct_original*100./total, correct_onnx*100./total, correct_coreml*100./total
    
    data_writer_run.writerow([model_short_name,training_id,framework,model_size_original, model_size_onnx, model_size_coreml, t_elapsed_conversion_onnx, t_elapsed_conversion_coreml, t_elapsed_saving_time_coreml, t_elapsed_original, load_time_onnx, load_time_coreml, round(accuracy_original*100/total_,4), round(accuracy_onnx*100/total_,4), round(accuracy_coreml*100/total_,4), round(miss_original_onnx*100/total_datasets,2),  round(miss_original_coreml*100/total_datasets,2),mean_abs_onnx,mean_abs_coreml, mean_rel_onnx, mean_rel_coreml,std_abs_onnx,std_abs_coreml, std_rel_onnx, std_rel_coreml, inference_time_original/total_datasets, inference_time_onnx/total_datasets, inference_time_coreml/total_datasets, '', '{:.0f}m {:.0f}s'.format(time_elapsed_overal_time // 60, time_elapsed_overal_time % 60)])
    
    
    
    
    

In [ ]:
import pandas as pd 

if not os.path.exists(onnx_path+framework+"/{}".format(model_short_name)):
            os.makedirs(onnx_path+framework+"/{}".format(model_short_name))
if not os.path.exists(coreml_path+framework+"/{}".format(model_short_name)):
    os.makedirs(coreml_path+framework+"/{}".format(model_short_name))
if not os.path.exists(error_path+framework+"/"):
    os.makedirs(error_path+framework+"/")
model_short_name = 'lstm'
framework = 'keras'

training_id = 3

for round_ in range(1):
    path = '/Volumes/Cisco/Summer2022/onnx-exchange/Train2/Amin/{}/{}/'.format(framework, model_short_name)
    since_0 = time.time()
    #model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
    #model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
    #model_name = 'tf_resnet18-cifar10_2021-10-29_{}'.format(training_id)
    model_name = 'keras-amin-lstm-3'
    since_orinal_0 = time.time()
    #model_name = 'tf_gru-imdb_2021-10-29_{}'.format(training_id)
    model = tf.keras.models.load_model(path+ model_name+'.h5')
    t_elapsed_original = time.time() - since_0
    model_size_original = os.path.getsize(path+ model_name+'.h5')
    #model_size_original

    since_1 = time.time()
    model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model,
                input_signature=None, opset=11, custom_ops=None,
                custom_op_handlers=None, custom_rewriter=None,
                inputs_as_nchw=None, extra_opset=None, shape_override=None,
                 target=None, large_model=False, output_path=onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))

    t_elapsed_conversion_onnx = time.time() - since_1


    since_1 = time.time()
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    load_time_onnx = time.time() - since_1
    model_size_onnx = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    onnx.checker.check_model(onnx_model)
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))


    since_1 = time.time()
    coreml_model = coremltools.convert(model)
    t_elapsed_conversion_coreml = time.time() - since_1
    since_1 = time.time()
    coreml_model.save(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    t_elapsed_saving_time_coreml = time.time() - since_1

    model_size_coreml = os.path.getsize(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))

    since_1 = time.time()
    coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}.mlmodel".format(model_short_name, model_name))
    load_time_coreml = time.time() - since_1

    if not os.path.exists(error_path+framework):
        Path(error_path+framework).mkdir(parents=True, exist_ok=True)

    print('round: ', round_)
    flag = 0
    if not os.path.exists(error_path+framework+"/runtime_accuracy_{}-v2.csv".format(model_short_name)):
        data_file_run = open(error_path+framework+"/runtime_accuracy_{}-v2.csv".format(model_short_name), mode='w', newline='', encoding='utf-8')
    else:
        data_file_run = open(error_path+framework+"/runtime_accuracy_{}-v2.csv".format(model_short_name), mode='a+', newline='', encoding='utf-8')
        flag = 1
    data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    if flag == 0: 
        data_writer_run.writerow(['model','training_id','framework','original_size', 'onnx_size', 'coreml_size','onnx_conversion_time', 'coreml_conversion_time', 'coreml_saving_time', 'original_load_time', 'onnx_load_time', 'coreml_load_time', 'accuracy_original', 'accuracy_onnx', 'accuracy_coreml', 'miss_original_onnx', 'miss_original_coreml','mean_abs_onnx','mean_abs_coreml', 'mean_rel_onnx', 'mean_rel_coreml','std_abs_onnx','std_abs_coreml', 'std_rel_onnx', 'std_rel_coreml', 'inference_time_original', 'inference_time_onnx', 'inference_time_coreml', '', 'overral_time'])
    _lets_convert(x_test, y_test, data_writer_run)
    data_file_run.close()

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 12152.37 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 94/94 [00:00<00:00, 1636.06 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 3782.79 ops/s]

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 1/1 [00:00<00:00, 529.99 ops/s]

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 72/72 [00:00<00:00, 102.59 ops/s]


round:  0
[[9.8360914e-01]
 [1.0000000e+00]
 [6.0989481e-05]
 ...
 [9.6239694e-07]
 [2.3447871e-03]
 [9.9999380e-01]]
[1. 1. 0. ... 0. 0. 1.] [0 1 1 ... 0 0 0]
accuracy_original:  21062 3938
accuracy_onnx:  21062 0


In [ ]:
np.sum(y_test == y_test)

In [11]:
path = '/Volumes/Cisco/Summer2022/onnx-exchange/Train2/Amin/{}/{}/'.format(framework, model_short_name)
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
#model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
#model_name = 'tf_resnet18-cifar10_2021-10-29_{}'.format(training_id)
model_name = 'amin-lstm'
since_orinal_0 = time.time()
#model_name = 'tf_gru-imdb_2021-10-29_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')

In [17]:
k_predict = model.predict(x_test.astype(np.float32))

In [24]:
y0 = convert_category(k_predict)
#correct_original = np.sum(y0 == y)
accuracy_original = model_scores(y_test.astype(np.float32), y0)
miss_test_original = get_miss_classification(y_test, y0)

In [25]:
accuracy_original*100/np.sum(y_test == y_test)

81.956

In [19]:
miss_test_original

4511

In [11]:
since_1 = time.time()
onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
load_time_onnx = time.time() - since_1
model_size_onnx = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
onnx.checker.check_model(onnx_model)
ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))

In [12]:
ort_inputs = {ort_session.get_inputs()[0].name: x_test.astype(np.float32)}
ort_outs = ort_session.run(None, ort_inputs)

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: embedding_2_input for the following indices
 index: 1 Got: 80 Expected: 300
 Please fix either the inputs or the model.